In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install ctranslate2 transformers[torch]
!pip install hf_hub_ctranslate2>=1.0.3 ctranslate2>=3.13.0
!pip install huggingface-hub

In [ ]:
from hf_hub_ctranslate2 import MultiLingualTranslatorCT2fromHfHub
from transformers import AutoTokenizer

from huggingface_hub import hf_hub_download

file_list = [
    'config.json',
    'generation_config.json',
    'model.bin',
    'sentencepiece.bpe.model',
    'shared_vocabulary.txt',
    'special_tokens_map.json',
    'tokenizer_config.json',
    'vocab.json'
]

for file_name in file_list:
    hf_hub_download(
        repo_id='michaelfeil/ct2fast-m2m100_418M',
        filename=file_name,
        local_dir='/tmp/model',
        repo_type='model'
    )

tokenizer = AutoTokenizer.from_pretrained('facebook/m2m100_418M')
tokenizer.save_pretrained('/tmp/model')

model = MultiLingualTranslatorCT2fromHfHub(
    model_name_or_path='/tmp/model',
    device='cpu',
    compute_type='int8',
    tokenizer=AutoTokenizer.from_pretrained('/tmp/model')
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/970M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!ls -lh /tmp/model

total 932M
-rw-r--r-- 1 root root 2.0K May 27 07:03 added_tokens.json
-rw-r--r-- 1 root root  159 May 27 07:03 config.json
-rw-r--r-- 1 root root  233 May 27 07:03 generation_config.json
-rw-r--r-- 1 root root 925M May 27 07:03 model.bin
-rw-r--r-- 1 root root 2.4M May 27 07:03 sentencepiece.bpe.model
-rw-r--r-- 1 root root 1.2M May 27 07:03 shared_vocabulary.txt
-rw-r--r-- 1 root root 1.6K May 27 07:03 special_tokens_map.json
-rw-r--r-- 1 root root  20K May 27 07:03 tokenizer_config.json
-rw-r--r-- 1 root root 3.6M May 27 07:03 vocab.json


In [ ]:
import time

start_time = time.time()

text = [
    'London is the capital and largest city of England, and the United Kingdom, with a population of around 8.8 million, ' +
    'and the largest city in Western Europe by metropolitan area, with a population of 14.8 million. ',

    'It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea and ' +
    'has been a major settlement for nearly two millennia. ',

    'The City of London, its ancient core and financial centre, ' +
    'was founded by the Romans as Londinium and retains its medieval boundaries. ',

    'The City of Westminster, to the west of the City of London, has for centuries hosted the national government and parliament. ',

    'Since the 19th century, the name "London" also refers to the metropolis around this core, ' +
    'historically split among the counties of Middlesex, Essex, Surrey, Kent, and Hertfordshire, ' +
    'which since 1965 has largely comprised Greater London, which is governed by 33 local authorities and ' +
    'the Greater London Authority.'
]

outputs = model.generate(
    text,
    src_lang=['en'] * 5,
    tgt_lang=['de'] * 5
)

duration = time.time() - start_time

print('')

print(f'Duration: {round(duration, 2)} seconds')

print('')

print(outputs)

# 13 - 25 seconds - CPU


Duration: 20.36 seconds

['London ist die Hauptstadt und größte Stadt Englands, und das Vereinigte Königreich, mit einer Bevölkerung von rund 8,8 Millionen, und die größte Stadt in Westeuropa durch die Metropolregion, mit einer Bevölkerung von 14,8 Millionen.', 'Es steht auf dem Thames River im südöstlichen Englands an der Spitze einer 50-Mile (80 km) Estuarie bis zum Nordsee und ist seit fast zwei Jahrtausenden eine große Siedlung.', 'Die Stadt London, ihr altes Kern und Finanzzentrum, wurde von den Römern als Londinium gegründet und bewahrt ihre mittelalterlichen Grenzen.', 'Die Stadt Westminster, im Westen der Stadt London, hat seit Jahrhunderten die nationale Regierung und das Parlament gehalten.', 'Seit dem 19. Jahrhundert bezieht sich der Name "London" auch auf die Metropole um diesen Kern, historisch unterteilt zwischen den Bezirken von Middlesex, Essex, Surrey, Kent und Hertfordshire, die seit 1965 weitgehend Groß London, die von 33 lokalen Behörden und der Groß Londoner Behör